In [1]:
import nengo
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import tensorflow as tf
import os
from nengo.dists import Choice
from datetime import datetime
# from nengo_extras.data import load_mnist
import pickle
from nengo.utils.matplotlib import rasterplot

import time

from InputData import PresentInputWithPause

# from nengo_extras.graphviz import net_diagram

from nengo.neurons import LIFRate

from nengo.params import Parameter, NumberParam, FrozenObject
from nengo.dists import Choice, Distribution, get_samples, Uniform

from nengo.utils.numpy import clip, is_array_like
from utilis import *
import keras

from args_mnist import args as my_args
import itertools
import random
import logging

In [10]:
(image_train, label_train), (image_test, label_test) = (keras.datasets.mnist.load_data())

In [11]:
input_nbr = 10

In [13]:
image_train_filtered = []
label_train_filtered = []
for i in range(0,input_nbr):
      
    image_train_filtered.append(image_train[i])
    label_train_filtered.append(label_train[i])

image_train_filtered = np.array(image_train_filtered)
label_train_filtered = np.array(label_train_filtered)

image_train_filtered = image_train_filtered/255

In [26]:
input_neurons_args = {
        "n_neurons":784,
        "dimensions":1,
        "label":"Input layer",
        "encoders":nengo.dists.Choice([[1]]),
        # "max_rates":nengo.dists.Uniform(22,22),
        # "intercepts":nengo.dists.Uniform(0,0),
        "gain":nengo.dists.Choice([2]),
        "bias":nengo.dists.Choice([0.6]),
        "neuron_type":MyLIF_in(tau_rc=0.1,min_voltage=-1, amplitude=1,tau_ref=0.002)
        # "neuron_type":nengo.neurons.SpikingRectifiedLinear()#SpikingRelu neuron. 
}

#Layer 1 parameters
layer_1_neurons_args = {
        "n_neurons":10,
        "dimensions":1,
        "label":"Layer 1",
        "encoders":nengo.dists.Choice([[1]]),
        "gain":nengo.dists.Choice([2]),
        "bias":nengo.dists.Choice([0]),
        # "intercepts":nengo.dists.Choice([0]),
        # "max_rates":nengo.dists.Choice([args.rate_out,args.rate_out]),
        # "noise":nengo.processes.WhiteNoise(dist=nengo.dists.Gaussian(0, 0.5), seed=1), 
        # "neuron_type":nengo.neurons.LIF(tau_rc=args.tau_out, min_voltage=0)
        # "neuron_type":MyLIF_out(tau_rc=args.tau_out, min_voltage=-1)
        "neuron_type":STDPLIF(tau_rc=0.1, min_voltage=-1, spiking_threshold=1, inhibition_time=2,tau_ref=0.002)
}
presentation_time = 0.35
pause_time = 0


learning_args = {
        "lr": 1,
        "winit_min":0,
        "winit_max":1,
        "vprog":0, 
        "vthp":0.5,
        "vthn":0.5,
        "vprog_increment":0,
        "voltage_clip_max":1.8,
        "voltage_clip_min":-1.8,
        "Vapp_multiplier":1,
}

In [27]:
model = nengo.Network("My network", seed = 1)
    #############################
    # Model construction
    #############################
with model:
    # picture = nengo.Node(PresentInputWithPause(images, presentation_time, pause_time,0))
    picture = nengo.Node(nengo.processes.PresentInput(image_train_filtered, presentation_time=presentation_time))
    true_label = nengo.Node(nengo.processes.PresentInput(label_train_filtered, presentation_time=presentation_time))
    # true_label = nengo.Node(PresentInputWithPause(labels, presentation_time, pause_time,-1))

    # input layer  
    input_layer = nengo.Ensemble(**input_neurons_args)
    input_conn = nengo.Connection(picture,input_layer.neurons,synapse=None)

    #first layer
    layer1 = nengo.Ensemble(**layer_1_neurons_args)

    #Weights between input layer and layer 1
    w = nengo.Node(CustomRule_post_v2_tio2(**learning_args), size_in=784, size_out=10)
    nengo.Connection(input_layer.neurons, w, synapse=None)
#         nengo.Connection(w, layer1.neurons, synapse=args.synapse_layer_1)
    # nengo.Connection(w, layer1.neurons,transform=g_max, synapse=None)
    # init_weights = np.random.uniform(0, 1, (n_neurons, n_in))
    # conn1 = nengo.Connection(input_layer.neurons,layer1.neurons,learning_rule_type=VLR(learning_rate=args.lr,vprog=args.vprog, vthp=args.vthp,vthn=args.vthn),transform=init_weights)

    #Lateral inhibition
    # inhib = nengo.Connection(layer1.neurons,layer1.neurons,**lateral_inhib_args) 

    #Probes
    # p_true_label = nengo.Probe(true_label, sample_every=probe_sample_rate)
    p_input_layer = nengo.Probe(input_layer.neurons)
    p_layer_1 = nengo.Probe(layer1.neurons)
    # weights_probe = nengo.Probe(conn1,"weights",sample_every=probe_sample_rate)

#         weights = w.output.history

        

    # with nengo_ocl.Simulator(model) as sim :   
with nengo.Simulator(model, dt=0.001, optimize=True) as sim:

        
    w.output.set_signal_vmem(sim.signals[sim.model.sig[input_layer.neurons]["voltage"]])
    w.output.set_signal_out(sim.signals[sim.model.sig[layer1.neurons]["out"]])


    sim.run((presentation_time+pause_time) * label_train_filtered.shape[0])

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.